In [144]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [145]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [146]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [147]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView("home_sales")



In [148]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql import functions as F

four_beds_df = home_df.filter(home_df.bedrooms == 4)

avg_four_beds_df = four_beds_df.groupBy("date_built") \
    .agg(F.round(F.avg("price"),2).alias("avg_four_bed"))\
    .orderBy(F.desc("date_built"))

query = """
    SELECT date_built,
    ROUND(AVG(price), 2) AS avg_four_bed
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built DESC
"""
spark.sql(query).show()

avg_four_beds_df.show()


+----------+------------+
|date_built|avg_four_bed|
+----------+------------+
|      2017|   296576.69|
|      2016|   296050.24|
|      2015|   307908.86|
|      2014|   299073.89|
|      2013|   299999.39|
|      2012|   298233.42|
|      2011|    302141.9|
|      2010|   296800.75|
+----------+------------+

+----------+------------+
|date_built|avg_four_bed|
+----------+------------+
|      2017|   296576.69|
|      2016|   296050.24|
|      2015|   307908.86|
|      2014|   299073.89|
|      2013|   299999.39|
|      2012|   298233.42|
|      2011|    302141.9|
|      2010|   296800.75|
+----------+------------+



In [149]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

three_bed_and_bath_df = home_df.filter((home_df.bedrooms == 3) & (home_df.bathrooms == 3))

avg_three_year_built_df = three_bed_and_bath_df.groupBy("date_built") \
    .agg(F.round(F.avg("price"), 2).alias("avg_3_bed_and_bath"))\
    .orderBy(F.desc("date_built"))

query = """
    SELECT date_built,
    ROUND(AVG(price), 2) AS avg_3_bed_and_bath
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built DESC
"""

spark.sql(query).show()

avg_three_year_built_df.show()

+----------+------------------+
|date_built|avg_3_bed_and_bath|
+----------+------------------+
|      2017|         292676.79|
|      2016|         290555.07|
|      2015|          288770.3|
|      2014|         290852.27|
|      2013|         295962.27|
|      2012|         293683.19|
|      2011|         291117.47|
|      2010|         292859.62|
+----------+------------------+

+----------+------------------+
|date_built|avg_3_bed_and_bath|
+----------+------------------+
|      2017|         292676.79|
|      2016|         290555.07|
|      2015|          288770.3|
|      2014|         290852.27|
|      2013|         295962.27|
|      2012|         293683.19|
|      2011|         291117.47|
|      2010|         292859.62|
+----------+------------------+



In [150]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

two_floors_df = home_df.filter(
    (home_df.bedrooms == 3) &
    (home_df.bathrooms == 3) &
    (home_df.floors == 2) &
    (home_df.sqft_living >= 2000))

avg_two_floors_df = two_floors_df.groupBy("date_built") \
    .agg(F.round(F.avg("price"), 2).alias("average_price"))\
    .orderBy(F.desc("date_built"))

query = """
    SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built DESC
"""

spark.sql(query).show()

avg_two_floors_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [151]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

view_df = home_df.filter(home_df.price >= 350000)

avg_price_view_df = view_df.groupBy("view") \
    .agg(F.round(F.avg("price"), 2).alias("average_price")) \
    .orderBy(F.desc("view"))


query = """
    SELECT view,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""

spark.sql(query).show()

avg_price_view_df.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|  

In [152]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [153]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [154]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

avg_price_view_df = spark.table("home_sales").filter(
    F.col("price") >= 350000).groupBy("view") \
    .agg(F.round(F.avg("price"), 2).alias("average_price")) \
    .orderBy(F.desc("view"))

query = """
    SELECT view,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""

spark.sql(query).show()

avg_price_view_df.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|  

In [155]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partition")


In [156]:
# 11. Read the parquet formatted data.
p_home_df_p=spark.read.parquet('date_built_partition')

p_home_df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [157]:
# 12. Create a temporary table for the parquet data.
p_home_df_p.createOrReplaceTempView('p_date_built_p')

In [158]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

query_parquet = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM p_date_built_p
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
"""
query_result = spark.sql(query_parquet)

query_result.show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 9.34600830078125e-05 seconds ---


In [159]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [160]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False